# DRA model & Ram Deck
## The GRP Model for Parametric Recurrent Event Data Analysis
[Quoted from ref 1. Fix typo in type II eq.]
In this model, the concept of virtual age is introduced. Denote by t1,t2,...,tn the successive failure times and let x1,x2,...,x3 represent the time between failures. Assume that after each event, actions are taken to improve the system performance. Let q be the action effectiveness factor. There are two GRP models.
 
Type I:  $v_i = v_{i-1} + q \cdot x_i = q \cdot t_i$
 
Type II: $v_i = q \cdot (v_{i-1} +  x_i )= \sum_{k=1}^i q^{i-k+1} \cdot x_k$

where $v_i$ is the virtual age of the system right after the ith repair. The Type I model assumes that the ith repair cannot remove the damage incurred before the ith failure. It can only reduce the additional age $x_i$ to $q \cdot x_i$. The Type II model assumes that at the ith repair, the virtual age has been accumulated to $v_{i-1} + x_i$. The ith repair will remove the cumulative damage from both current and previous failures by reducing the virtual age to $q \cdot (v_{i-1} + x_i )$.

The power law function is used to model the rate of recurrence, which is: $\lambda(t) = \lambda\beta t^{\beta-1}$

The conditional pdf is $f(t_i|t_{i-1}) = \lambda \beta (x_i + v_{i-1})^{\beta-1} \cdot e^{-\lambda[(x_i + v_{i-1})^\beta - V_{i-1}^\beta]}$

Comparing the ROCOF with Weibull hazard function, we can get: $\lambda = \frac{1}{\eta^\beta}$ or $\eta = (\frac{1}{\lambda})^{\frac{1}{\beta}}$

## Approximation

There is no close form for the renewal function E(N(t)) of General Renewal Process. Monte Carlo simulation can be used but it is desirable to find an approximation equation of renewal function to feed into the RAM Deck. 

In [1]:
# preparing the code
import matplotlib.pyplot as plt
import numpy as np
import time # only to track running time.
# x = ro.r('rweibull(n,shape = 2, scale)')
# x = scale * np.random.weibull(shape, n)

# use Bokeh for visulisation
from bokeh.io import output_file,  output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh import palettes

# a little preparation for interpolation
from scipy.interpolate import interp1d
from math import log10, floor

## Simulation based on RAD analysis

Port the code from **Ken's** matlab to python for speed.

Define funtion simRDA for simulation of type 1 renewal process. 
- Generating time to first failure - Weibull
- Generating time to ith failure, conditional Weibull - inverse CDF
- Type 1 general renewal process, $age_{virtual} = q \cdot time_i$

In [2]:
def simRDA(alpha, beta, q, n_failure, n_sim):
    n_failure = int(n_failure)
    n_sim = int(n_sim)
    print(type(n_failure))
    ttf = np.empty([n_failure, n_sim])
    v_time = np.empty(n_sim)

# generate time to 1st failure, scale = alpha, shape = beta
    ttf[0,:] = np.random.weibull(beta, n_sim) * alpha
    v_time = ttf[0, :] * q

# generating time to ith failure - inverse CDF
    for i in range(1, n_failure) :
        ttf[i, : ] = alpha * (((v_time/alpha) ** beta - np.log(1-np.random.random_sample(n_sim))) **(1. /beta)) - v_time + ttf[i-1, : ]
        v_time = q * ttf[i, : ]
    return ttf

### Test the simulation 
The simulation is faster than same code in R or matlab.

### Data Analysis of renewal process
Arrival time is the time nth event happens. E(nth Arrival time) and the Renewal function E(N(t)) is not same.

In [4]:
E_arrival_time = np.mean(ttf, axis=1)
print(E_arrival_time)
# prob(count = n, t<time)
def prob(count, time, ttf):
    nth = ttf[count-1, :]
    return nth[nth < time].size/ttf.size
print(prob(2, alpha, ttf) + prob(1,alpha, ttf))

# prob(count < n, t<time)
def cumProb(count, time, ttf):
    to_n = ttf[0:count, :]
    return to_n[to_n < time].size/ttf.size
print(cumProb(2, alpha, ttf))

[  44276.24029249   73797.62082974   97215.25034356  117103.20022985
  134655.16235328  150486.43723699  165032.54008947  178548.94482681
  191227.22000194  203204.51376036  214567.48191559  225395.70596781
  235769.11732534  245735.21406676  255347.43369508  264614.77250668
  273602.56932296  282326.47429279  290809.62507564  299055.02454507
  307099.28530251  314943.26820281  322599.02083137  330079.54069514
  337398.59899693  344594.1242844   351634.7296359   358532.28587309
  365309.62648392  371962.36345461  378514.07731002  384946.24532968
  391286.6531885   397532.9154637   403688.46454963  409735.10006819
  415709.36552236  421602.63905134  427422.58967384  433144.31610918]
0.020747474999999998
0.020747475


### Calculate Renewal Function based on simulation results
For counting process, N ≡ {N(t) : t ≥ 0},  Renewal function m(t) ≡ E[N(t)].

As we can see from above, the E(N(t)) at average arrival time of nth failure do not equal to n. 

### Ken's proposed approximation

#### Full Repair_Deterministic
Based on Ref[2]: $M[i] = F[i]+ \sum_{i=0}^{n} ( M[n-i] \cdot cdf_d[i])$

#### Minimal Repair_Deterministic
$H = (\frac{t}{\alpha})^\beta$; 

#### Partial Repair_Deterministic (Proposed)
$$MH = q \cdot H+ (1-q) \cdot M$$

In [21]:
def H_t(alpha, beta, time, n_step=1000):
    time_step = np.linspace(0, time, n_step)
    return  (time_step/alpha) ** beta

def M_t(alpha, beta, time, n_step=1000):
    time_step = np.linspace(0, time, n_step)
    cdf = 1 - np.exp(- (time_step / alpha) ** beta)
    d_cdf = np.concatenate(([cdf[0]], np.diff(cdf)), axis=0)
# Full Repair_Deterministic
    m = np.zeros(n_step) 
    for i in range(n_step-1):
        m[i+1] = cdf[i+1] + np.sum( np.multiply(m[i::-1], d_cdf[0:i+1]))
    return m

def mh(alpha, beta, q, time, n_step=1000):
    return q * H_t(alpha, beta, time, n_step) + (1 - q) * M_t(alpha, beta, time, n_step)

mht = mh(alpha, beta, q, max(E_arrival_time))
print(mht[mht.size-1])

# Clean the data before moving to next secction
del alpha, beta, q, ttf, E_arrival_time

42.176018189


### Compare Simulation vs. Approximation
To compare the goodness of fit of the proposed $M(t)$ function, we try different shape factors c(0.5, 1, 1.5, 2, 4). Q will be set to c(0, 0.25, 05, 075, 1) for each shape factor. The scale factor will be set to 50000. 

In [ ]:
# Run the simulation and store the arrivial time, and renewal function E(N(t))

# Define parameters
q_list = [0, 0.25, 0.5, 0.75, 1]
#q_list = [0.5, 0.75]
beta_list = [0.5, 1, 1.5, 2, 3, 4]
#beta_list = [ 1.5, 2]
alpha = 50000
n_sim = 1e6
n_fail = 40
maxtime = 1e6
n_step = 1000
time_step =  np.linspace(0, maxtime, n_step)

# start clock
start_time = time.time()

paras = []
mt = []
ht = []
mht = []
ent = []
E_arrival = []
# mt, ht are function of (alpha, beta). The results are stored in a list of len(beta_list)
# mht, ent, arrivail time are function of (beta, q). The results are stored in a nested list [len(beta), [len(q)]]

for i in range(len(beta_list)):
    beta = beta_list[i]
    mt.append( M_t(alpha, beta, maxtime, n_step))
    ht.append( H_t(alpha, beta, maxtime, n_step))
    mht_b = []
    ent_b = []
    E_arrival_b = []
    for ii in range(len(q_list)):
        q = q_list[ii]
        paras.append([beta, q])
        ttf = simRDA(alpha, beta, q, n_fail, n_sim)
        mht_b.append(mh(alpha, beta, q, maxtime))
        E_arrival_b.append(np.mean(ttf, axis=1))
        ent_b.append(E_Nt(E_arrival_b[ii], ttf))        
        del ttf
    mht.append(mht_b)
    E_arrival.append(E_arrival_b)
    ent.append(ent_b)
print("--- %s seconds ---" % (time.time() - start_time))

<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>


In [ ]:
# Visualization goes here
# Parameters from last cell

fig = []


for i in range(len(beta_list)):
    beta = beta_list[i]
    p = figure(title = "\u03B2 = " + str(beta), x_axis_label='time', y_axis_label='Expected # of Repairs', y_range=(0,n_fail))
    p.line(time_step, mt[i], legend="Replace", line_width=4, line_color="red")
    p.line(time_step, ht[i], legend="Minimal Repair", line_width=4, line_color="black")
    colors = palettes.viridis(len(q_list))
    for ii in range(len(q_list)):
        q = q_list[ii]
        p.line(time_step, mht[i][ii], line_color=colors[ii], legend = "q= " + str(q), line_width=2)
#        p.diamond(E_arrival_time, np.arange(n_fail)+1,  fill_color=colors[i], size=12)
        p.circle(E_arrival[i][ii], ent[i][ii], fill_color=colors[ii], size=8)
    p.legend.location = "bottom_right"
    fig.append(p)
#    exec("p" + str(i) + " = p")
#    print("p" + str(i) + " = p")
#    exec("fig.append(" + "p" + str(i) + ")")
#    print("fig.append(" + "p" + str(i) + ")")
gp = gridplot(*fig, ncols = 2)
output_file("simRDA_multi_Beta.html")
show(gp)


## Interpolation of the simulated result
The interp1d class in scipy.interpolate is a convenient method to create a function based on fixed data points which can be evaluated anywhere within the domain defined by the given data using linear interpolation. 
- Use simRDA to generate events history given the parameters
- Calculate E(N(t)) at points ...
- Interpolating (cubic)

In [ ]:
# define parameters
def round_to_1(x):
    return round(x, -int(floor(log10(abs(x)))))

alpha = 50000.
beta = 2.
q = 0.5
n_fail = 40
n_sim = 1e6
i_step = n_fail

start_time = time.time()

ttf = simRDA(alpha, beta, q, n_fail, n_sim)

E_arrival = np.mean(ttf, axis=1)
maxtime = round_to_1(max(E_arrival) * 1.2)

time_step = np.linspace(0, maxtime, i_step)

ent = E_Nt(time_step, ttf)

ent_interpf = interp1d(time_step, ent, kind='cubic')

time_new = np.linspace(0, maxtime, 1000)

p = figure(title = "Interpolated Renewal Function", x_axis_label='time', y_axis_label='Expected # of Repairs')
p.line(time_new, ent_interpf(time_new), legend="interpolation")
p.circle(time_step, ent, legend = "simulation")
p.legend.location = "bottom_right"
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
output_notebook()
output_file("simRDA_interp.html")
show(p, notebook_handle = True)

# to do
Given the parameter from data analysis $\lambda, \beta, q$, a one-million-run simuation to obtain the interpolation may take only several seconds on my i7 laptop. 

Check whether LDM database can take the above python code. 


# Reference
1. RDA, Weibull++; http://www.weibull.com/hotwire/issue59/relbasics59.htm
2. Saeed Maghsoodloo and Dilcu Helvaci, “Renewal and Renewal-Intensity Functions with Minimal Repair,” Journal of Quality and Reliability Engineering, vol. 2014, Article ID 857437, 10 pages, 2014. doi:10.1155/2014/857437
3. E. Smeitink and R. Dekker, "A simple approximation to the renewal function [reliability theory]," in IEEE Transactions on Reliability, vol. 39, no. 1, pp. 71-75, Apr 1990.

# Appendix
## Weibull Random Numbers can be generated by R or Python
The Weibull distribution with shape parameter a and scale parameter b has density given by

$f(t) = \frac{\beta}{\eta}(\frac{t}{\eta})^{\beta-1}e^{- (\frac{t}{\eta})^\beta}$ where $\eta$ is the scale factor, and $\beta$ is the shape factor.

The hazard function $h(t) = \frac{\beta} {\eta} (\frac{t} {\eta})^{\beta-1}$


For t > 0. The cumulative distribution function is $F(t) = 1 - e^{-(\frac{t}{\eta})^\beta}$.

### Use rpy2 package to call R random number generator: 
rweibull(n, shape, scale = 1)

> import rpy2.robjects as ro

> x = ro.r('rweibull(n, shape, scale =1)')

### Use numpy Weibull random number generator
numpy.random.weibull(a, size=None)
Draw samples from a Weibull distribution.

Draw samples from a 1-parameter Weibull distribution with the given shape parameter a.

$X = (-\ln(U))^\frac{1}{a}$

Here, U is drawn from the uniform distribution over (0,1].

The more common 2-parameter Weibull, including a scale parameter $\lambda$ is just $X = \lambda(-\ln(U))^\frac{1}{a}$.

Note to self, python index start at 0.

In [11]:
#scrapbook window
